In [1]:
import wrds
import pandas as pd
import datetime
import numpy as np
import pickle
from time import time

In [ ]:
db = wrds.Connection()

In [ ]:
q = ("select a.gvkey,a.latest,b.cshoq,b.prccq,b.mkvaltq,b.cshoq*b.prccq as market_cap,b.curcdq "
     "from "
        "(select gvkey,max(datadate) as latest "
         "from "
         "compm.fundq where datadate > '2017-01-01' " 
         "group by gvkey) a inner join "
             "(select gvkey,datadate,mkvaltq,cshoq,prccq,curcdq "
                "from compm.fundq where cshoq>0 and prccq>0 and curcdq='USD') b "
    "on a.gvkey = b.gvkey and a.latest=b.datadate "
     "order by market_cap desc "
    "limit 100")

In [ ]:
a = db.raw_sql(q)
top_2000_eq_gvkey = a['gvkey'].values.tolist()
top_2000_eq_gvkey = tuple(["'%s'"%str(i) for i in top_2000_eq_gvkey])
top_2000_eq_gvkey = ",".join(top_2000_eq_gvkey)

In [ ]:
q2 = ("select datadate,gvkey,tic,saleq,cogsq,xsgaq,oiadpq,niq,"
      "cheq, rectq, invtq, acoq, ppentq, aoq, dlcq, apq, txpq, lcoq, ltq, dlttq,cshoq "
    "from compm.fundq "
     "where gvkey in (%s) ")%top_2000_eq_gvkey
c = db.raw_sql(q2)
c.shape

In [ ]:
blnc_sheet_list = ['cheq','rectq','invtq','acoq','ppentq','aoq',
                                'dlcq','apq','txpq','lcoq','ltq','dlttq','cshoq']
mqr_list = [x + '_mqr' for x in blnc_sheet_list]

In [ ]:
income_list = ['saleq','cogsq','xsgaq','oiadpq','niq']
x = c[['gvkey','datadate'] + income_list + blnc_sheet_list]
x = x[x['gvkey'] == '006066']
x.sort_values('datadate').head()

In [ ]:
t1 = time()
%run wrds_data_processing.py
print time()-t1

In [ ]:
qq = ("select gvkey,datadate,prccm "
     "from compm.secm "
     "where gvkey='160329' ")
d = db.raw_sql(qq).sort_values('datadate')
d = d.set_index('datadate')

In [ ]:
d.loc[datetime.date(2014,3,31)]

In [ ]:
qq = ("select * "
     "from compm.secm "
     "where gvkey='001690' ")
d = db.raw_sql(qq).sort_values('datadate')
d = d.set_index('datadate')

In [ ]:
d.loc[datetime.date(2014,3,31)]

In [ ]:
d.to_pickle("prccm.pkl")

In [ ]:
q5 = ("select gvkey,datadate,split "
     "from compm.sec_split "
     "where gvkey='160329' ")
e = db.raw_sql(q5).sort_values('datadate')

In [ ]:
e= pd.read_pickle('prccm.pkl')

In [1]:
from time import time

In [2]:
t1 = time()
%run build_data.py
print time()-t1

Enter your WRDS username [lakshay]:dsanghi
Enter your password:········
WRDS recommends setting up a .pgpass file.
You can find more info here:
https://www.postgresql.org/docs/9.5/static/libpq-pgpass.html.
Loading library list...
Done
Shape of raw dataframe: 13979,21


153.779999971
